<a href="https://colab.research.google.com/github/ZiyueNie/FM/blob/master/src/ZiyueNie_Midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

__Name__

__Rules__ 

- After finish, you should email to me PDF version with file name "firstname_lastname_midterm.pdf"
- Any kind of communications are strictly prohibited. But you can use book, note, or internet.
- All your answers shall be rigourously justified.
- No python modules shall be imported except those given below

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.integrate import quad
from scipy.optimize import fmin

__Problem__

We consider BSM model with the following parameters:

In [0]:
'''======parameters============='''
s0, r, sigma, T, K, otype = [30., .02, 0.3, 1., 31.0, 1] # spot, rate, vol, maturity, strike, option type

- (Q) Write for the european call the explicit pricing formula, then find call price.



<br>For European call option,
$$C_0 = S_0  \Phi(d_1) - K e^{-rT} \Phi(d_2),$$
where $d_i$ are given as
$$d_1 = \frac{(r + \frac 1 2 \sigma^2) T - \ln \frac{K}{S_0}}{\sigma \sqrt T},$$
and
$$d_2 = \frac{(r - \frac 1 2 \sigma^2) T - \ln \frac{K}{S_0}}{\sigma \sqrt T} = d_1 - \sigma \sqrt T$$


In [11]:
'''========
Black-Scholes-Merton formula. 
=========='''

def bsm(s0, r, sigma, T, K, otype):
    d1 = (np.log(s0 / K) + (r + 0.5 * sigma ** 2) 
          * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    return (otype * s0 * norm.cdf(otype * d1) 
            - otype * np.exp(-r * T) * K * norm.cdf(otype * d2))
price=bsm(30, 0.02, 0.3, 1., 31.0, 1)

    
print('>>>> your answer below: ')
print(str(price))


>>>> your answer below: 
3.4102506629837706


In [0]:
def bsm_price(self, vanilla_option):
    s0 = self.init_state
    sigma = self.vol_ratio
    r = self.drift_ratio
    
    otype = vanilla_option.otype
    k = vanilla_option.strike
    maturity = vanilla_option.maturity
    
    d1 = (np.log(s0 / k) + (r + 0.5 * sigma ** 2) 
          * maturity) / (sigma * np.sqrt(maturity))
    d2 = d1 - sigma * np.sqrt(maturity)
    
    return (otype * s0 * norm.cdf(otype * d1) #line break needs parenthesis
            - otype * np.exp(-r * maturity) * k * norm.cdf(otype * d2))

Gbm.bsm_price = bsm_price


In [6]:
def bsm(s0, r, sigma, T, K, otype):
    d1 = (np.log(s0 / K) + (r + 0.5 * sigma ** 2) 
          * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    return (otype * s0 * norm.cdf(otype * d1) #line break needs parenthesis
            - otype * np.exp(-r * T) * K * norm.cdf(otype * d2))
bsm(30, 0.02, 0.3, 1., 31.0, 1)

3.4102506629837706

- (Q) 
    - write Fourier transform $\phi$ of $\ln S_T$;
    - write down the call price in terms of $\phi$;
    - then find call price using the above formula.

(your latex here)

In [0]:
'''========
Black-Scholes-Merton price by Fourier transform. 
=========='''

def bsm_fourier(s0, r, sigma, T, K, otype):
    pass

print('>>> your answer below ')

>>> your answer below 


- (Q) Prove that call price is increasing with respect to its maturity, and demonstrate your result numerically.

(your latex here)

In [0]:
'''====your code here==='''

pass

- (Q) 
Do you think that call price is also increasing with respect to its maturity for Heston model? Please justify your answer.

(your latex here)

- (Q) With the same parameters for GBM above, price the arithmetic asian option with $T = 1$, $K= 31$, and two periods (i.e. average prices of three time instants)

In [0]:
'''====your code here==='''

pass

__Problem__

We consider the call price underlying XYZ stock with spot price $\$30$ and interest rate $2\%$. Below is the current market call prices corresponding to its maturity $T=1$ and different strikes:

In [0]:
s0, r, T, otype = [30., .02, 1., 1] # spot, rate, maturity, option type
strike_arr = [28.0, 28.5, 29.0, 29.5, 30.0, 30.5, 31.0, 31.5, 32.0] # strike array
market_call_arr = [5.7, 5.0, 4.5, 4.1, 3.8, 3.6, 3.6, 3.7, 3.9] # corresponding market price

- (Q) Find their implied volatilities and draw its volatility smile

In [0]:
'''====your code here===='''

pass

- (Q.) Find bsm calibrated volatility to the above data by minimizing $L^2$ norm in the error.

In [0]:
'''====your code here===='''

pass

__Problem__

Let $f:\mathbb R\mapsto \mathbb R$ be a smooth even function 
satisfying $f(0) = 0$. 
Our objective is to approximate  the second order derivative $f''(0)$. 
- (Q.) Prove that $f'(0) = 0.$
- (Q.) Ziyue proposes the following estimator for $f''(0)$: for a step size $h$
 $$a_{h} = \frac{2f(h)}{h^{2}}.$$
 Please justify that Ziyue's estimation has its convergence $O(h^{2})$.
- (Q.) Is there anyway to improve the above 
 convergence to $O(h^{4})$ in the form of 
 $$b_{h} = \frac{c_{1} f(h) + c_{2}f(2h)}{h^{2}}$$
 for some constants $c_{1}$ and $c_{2}$?
- (Q.) If the above function $f$ is odd and other properties remain the same, how do you want to find the $f''(0)$ efficiently?


__Soln__ (your latex input here)


__Problem__

Thomas proposed the following estimator for the value $\pi$.
$$\hat \pi = c \cdot I(X_1^2 + X_2^2< 1/2)$$
where $X_1$ and $X_2$ are two independent uniform random variables on $(-1,1)$.

- (Q) find the constant $c$ so that $\hat \pi$ is unbiased estimator to $\pi$;
- (Q) for $c?$ in your above answer, what is its MSE?

(your latex here)


- (Q) can you propose a better mc estimator with smaller MSE?

__Soln__ (your latex here)

- (Q) can you demonstrate the theoretical MSE obtained in your algorithm above numerically via python code?

In [0]:
'''====== your code here ======='''
pass

__Problem__

- (Q) find the conditional probability of a random varialbe $Z$ in $B_{1/2}$,  given that $Z$ is uniformly distributed on $B_1$.

__Soln__(your latex here)


- (Q) Demonstrate your answer numerically by doing so:
    - Generate 1000 samplings uniformly from $B_1$;
    - count the points in $B_{1/2}$. 
    - find the ratio of number of points in the small ball and the big ball.

In [0]:
'''===== your code here======'''
num_sample = 1000
sample_arr = np.zeros([2,num_sample])

pass

- (Q) Justify your random sampling in the previous question

__soln__ (your latex here)

